In [2]:
from google.cloud import bigquery
from google.cloud import bigquery_storage

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Create BigQuery client
client = bigquery.Client()

# Create BigQuery Storage client
bqstorage_client = bigquery_storage.BigQueryReadClient()

query = """
SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE date <= '2023-03-31' and date >= '2023-01-01'
"""

# Fetch results using BigQuery Storage API
df = client.query(query).to_dataframe(bqstorage_client=bqstorage_client)

# Display results
df.head()

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,RINV-04540800134,2023-03-08,2619,HY-VEE WINE AND SPIRITS / WDM,1725 74TH ST,WEST DES MOINES,50266,None,None,POLK,...,35318,BARTON VODKA,6,1750,7.06,10.59,-6,-63.54,-10.50,-2.77
1,INV-57107700003,2023-03-30,4599,QUIK TRIP #500 / HUBBELL DM,3700 HUBBELL AVE,DES MOINES,50317,POINT(-93.54489 41.62901),None,POLK,...,11771,BLACK VELVET MINI,8,50,8.87,13.31,1,13.31,0.05,0.01
2,INV-55463200007,2023-01-31,2643,HY-VEE WINE AND SPIRITS / WATERLOO,2126 KIMBALL AVE,WATERLOO,50701,POINT(-92.35698 42.47029),None,BLACK HAWK,...,15621,JAMESON MINI,12,50,9.90,14.85,1,14.85,0.05,0.01
3,INV-56976600004,2023-03-27,6119,ALLCOOL LIQUORS / WDM,5904 ASHWORTH RD,WEST DES MOINES,50266,POINT(-93.78898 41.58439),None,POLK,...,18375,FOUR ROSES SMALL BATCH SELECT MINI,5,50,41.80,62.70,1,62.70,0.05,0.01
4,INV-55400000018,2023-01-30,6226,ROCK C LIQUOR & WINE & TOBACCO / IOWA FALLS,418 ROCKSYLVANIA AVENUE,IOWA FALLS,50126,POINT(-93.26263 42.52143),None,HARDIN,...,25601,SEAGRAMS 7 CROWN MINI,12,50,6.17,9.26,1,9.26,0.05,0.01


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624100 entries, 0 to 624099
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   invoice_and_item_number  624100 non-null  object 
 1   date                     624100 non-null  dbdate 
 2   store_number             624100 non-null  object 
 3   store_name               624100 non-null  object 
 4   address                  623386 non-null  object 
 5   city                     623386 non-null  object 
 6   zip_code                 623386 non-null  object 
 7   store_location           608433 non-null  object 
 8   county_number            0 non-null       object 
 9   county                   623386 non-null  object 
 10  category                 624100 non-null  object 
 11  category_name            624100 non-null  object 
 12  vendor_number            624100 non-null  object 
 13  vendor_name              624100 non-null  object 
 14  item

In [5]:
# Group by the category column and sum the bottles sold for each category
category_bottles = df.groupby('category_name')['bottles_sold'].sum().reset_index()

# Optionally, sort the results by bottles sold in descending order for easier viewing
category_bottles = category_bottles.sort_values(by='bottles_sold', ascending=False)
category_bottles.head(10)

,category_name,bottles_sold
9,AMERICAN VODKAS,1483592
42,WHISKEY LIQUEUR,1047475
12,CANADIAN WHISKIES,725525
37,STRAIGHT BOURBON WHISKIES,396239
36,SPICED RUM,376495
6,AMERICAN FLAVORED VODKA,241439
7,AMERICAN SCHNAPPS,238508
0,100% AGAVE TEQUILA,226999
10,BLENDED WHISKIES,205653
29,MIXTO TEQUILA,181434


In [7]:
category_bottles.shape

(44, 2)

In [15]:
category_bottles[category_bottles['bottles_sold']> 10000].shape

(38, 2)

In [10]:
vendor_bottles = df.groupby('vendor_name')['bottles_sold'].sum().reset_index().sort_values(by='bottles_sold', ascending=False)
vendor_bottles.shape

(206, 2)

In [11]:
vendor_bottles.head(10)

,vendor_name,bottles_sold
164,SAZERAC COMPANY INC,1986035
52,DIAGEO AMERICAS,990541
82,HEAVEN HILL BRANDS,554437
113,LUXCO INC,452347
97,JIM BEAM BRANDS,439795
68,FIFTH GENERATION INC,347398
145,PERNOD RICARD USA,317075
124,MCCORMICK DISTILLING CO.,240998
10,BACARDI USA INC,227436
62,E & J GALLO WINERY,217603


In [18]:
vendor_bottles[vendor_bottles['bottles_sold']<100]

,vendor_name,bottles_sold
105,KOLOA RUM COMPANY,96
144,PATRIARCH DISTILLERS LLC / SOLDIER VALLEY SPIRITS,94
183,TOM'S TOWN DISTILLING CO,87
138,OPICI IMPORT COMPANY,84
200,WMC-PBD LLC/ PINE BLUFFS DISTILLING,78
...,...,...
125,MEISENBURG BREWING & DISTILLING,6
140,PACIFIC EDGE WINE & SPIRITS,6
127,MIDDLE WEST SPIRITS,6
198,WILLIAM WRIGHT / BAY AND ROUSE LLC,6


In [19]:
df['city'].nunique()

451